In [4]:
import pandas as pd
import numpy as np
from sqlalchemy import *
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pymysql
from config import sqlpass
from flask import Flask, jsonify
import json

In [6]:
df = pd.read_csv("athlete_events.csv")
df1 = pd.read_csv("olympicslocations.csv")

In [8]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [9]:
df1.head()

,Games,Year,Season,City,lat,lon,all_games,num
0,1992 Winter,1992,Winter,Albertville,34.267849,-86.208801,1992 Winter,1
1,1928 Summer,1928,Summer,Amsterdam,52.370216,4.895168,1928 Summer,1
2,1920 Summer,1920,Summer,Antwerpen,51.219448,4.402464,1920 Summer,1
3,1896 Summer,1896,Summer,Athina,37.983810,23.727539,"1896 Summer, 1906 Summer, 2004 Summer",3
4,1906 Summer,1906,Summer,Athina,37.983810,23.727539,"1896 Summer, 1906 Summer, 2004 Summer",3


In [10]:
rds_connection_string = f"root:{sqlpass}@127.0.0.1/olympics_db?charset=utf8"
engine = create_engine(f'mysql://{rds_connection_string}', encoding='utf-8')

In [ ]:
df.to_sql(name='athlete_events', con=engine, if_exists='replace', index=False)

In [4]:

#these add the data to the SQL database, ONLY RUN ONCE
#COMMENT OUT AFTER THE DATA AS BEEN ADDED

df1.to_sql(name='locations', con=engine, if_exists='replace', index=False)


In [6]:
inspector = inspect(engine)
inspector.get_table_names()

['athletes', 'locations']

In [7]:
#gets all the name of the columns
columns = inspector.get_columns('locations')
for x in columns:
    print(x['name'], x["type"])

ID INTEGER(11)
Games VARCHAR(50)
Year INTEGER(11)
Season VARCHAR(50)
City VARCHAR(50)
lat FLOAT(10, 6)
lon FLOAT(10, 6)
all_games VARCHAR(50)
num INTEGER(11)


In [8]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Athletes = Base.classes.athletes
Locations = Base.classes.locations

In [9]:
Base.classes.keys()


['athletes', 'locations']

In [11]:
session = Session(engine)